In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine, inspect
import os

def get_bitcoin_data():
    start_date = '2023-12-31'
    end_date = datetime.now().date() - timedelta(days=1)
    url = f'https://web-api.coinmarketcap.com/v1/cryptocurrency/ohlcv/historical?id=1&convert_id=2781&time_start={start_date}&time_end={end_date}'
    file_path = 'C:/Users/Xande/Documents/extract_btc/bitcoin_data.csv'

    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f'Failed to fetch data: {response.status_code}')

    json_data = response.json()
    raw_data = json_data.get('data', {}).get('quotes', [])

    btc_data = [{key: quote['quote']['2781'][key] for key in quote['quote']['2781']} for quote in raw_data] # 2781 - код USD

    pd.DataFrame(btc_data).to_csv(file_path, sep=';', index= False)


In [ ]:
def load_btc_data_into_db():
    file_path = 'C:/Users/Xande/Documents/extract_btc/bitcoin_data.csv'

    # Получаем конфигурацию для подключения к БД из переменных окружения (если переменная не определена, используем значения по умолчанию)
    pg_host = os.getenv('PG_HOST', 'localhost')
    pg_port = os.getenv('PG_PORT', '5432')
    pg_user = os.getenv('PG_USER', 'postgres')
    pg_password = os.getenv('PG_PASSWORD', 'root')
    pg_database = os.getenv('PG_DATABASE', 'dwh_dbt')

    try:
        engine = create_engine(f'postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_database}')
        inspector = inspect(engine)
        if 'raw' not in inspector.get_schema_names():
            engine.execute('CREATE SCHEMA raw')

        df = pd.read_csv(file_path, sep=';')
        with engine.connect() as conn:
            df.to_sql('btc_data', conn, schema='raw', if_exists='replace', index=False)
        print('Данные успешно загружены в базу данных.')
    except Exception as e:
        raise RuntimeError(f'Произошла ошибка при загрузке данных в базу данных: {e}')


get_bitcoin_data()
load_btc_data_into_db()